# CAMBI

#### Source: https://github.com/Netflix/vmaf

CAMBI (Contrast Aware Multiscale Banding Index) is Netflix's detector for banding (aka contouring) artifacts.

The CAMBI score starts at 0, meaning no banding is detected. A higher CAMBI score means more visible banding artifacts (worse quality). The maximum CAMBI observed in a sequence is 24 (unwatchable). As a rule of thumb, a CAMBI score around 5 is where banding starts to become slightly annoying (also note that banding is highly dependent on the viewing environment - the brigher the display, and the dimmer the ambient light, the more visible banding is) (Netflix,2021). [See more](https://github.com/Netflix/vmaf/blob/master/resource/doc/cambi.md). 


### Example:

In this example the distorted video is a video in 640x360 resolution, encoded with x264 using a CRF value of 40. Reference and distored videos are converted to Y4M.

In [ ]:
!ffmpeg -hide_banner -y -i videos/ref/bbb_1920x1080.mp4 /tmp/ref.y4m
!ffmpeg -hide_banner -y -i videos/dist/x264/bbb_640x360_CRF40.mp4 -vf scale=1920x1080 /tmp/dist.y4m

Results are stored in XML format in the file `results/cambi/bbb_cambi.xml`:

In [ ]:
! mkdir -p results/cambi && \
PYTHONPATH=python /vmaf/libvmaf/build/tools/vmaf \
  --reference /tmp/ref.y4m \
  --distorted /tmp/dist.y4m \
  --no_prediction --feature cambi --output results/cambi/bbb_cambi.xml

#### Graphical representation.

In [ ]:
import pandas as pd
import json
import plotly.express as px
import xmltodict

# File with the metric data (adapt to your needs)
path_file='results/cambi/bbb_cambi.xml'

json_object = json.dumps(path_file) 

with open(path_file) as xml_file:
    data_dict = xmltodict.parse(xml_file.read())

str_data=json.dumps(data_dict)
data=json.loads(str_data)

cambi_frames=[]
for frame in data['VMAF']['frames']['frame']:
    cambi_frames.append(float(frame['@cambi']))

df=pd.DataFrame(cambi_frames, columns=['CAMBI'])

fig = px.line(df,x=df.index, y='CAMBI',
              title='CAMBI by frame',labels={"index": "Frame"}, template='simple_white',markers=True)

fig.add_hline(y=df.CAMBI.mean(), line_width=2, line_dash='dash')

fig.update_layout(
    title={
        'y':0.9,
        'x':0.5,
        'xanchor': 'center'},
    showlegend=True)

fig.show()